In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

import time
import requests
import zipfile
import os
import pandas as pd
import re
from bs4 import BeautifulSoup

In [2]:
def un_zip(file_name, to_dir='./'):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(to_dir):
        pass
    else:
        os.mkdir(to_dir)
    for names in zip_file.namelist():
        zip_file.extract(names, to_dir)
    zip_file.close()


def download_driver(to_dir='./', version=''):
    print('install chrome-driver first')
    url = 'http://npm.taobao.org/mirrors/chromedriver/LATEST_RELEASE'
    if len(version)>0:
        url = 'http://npm.taobao.org/mirrors/chromedriver/LATEST_RELEASE_'+version
        
    version = requests.get(url).content.decode('utf8')
    driver_file = 'http://npm.taobao.org/mirrors/chromedriver/' + version + '/chromedriver_win32.zip'
    r = requests.get(driver_file)
    download_zip = "chromedriver_win32.zip"
    with open(download_zip, "wb") as code:
        code.write(r.content)
    un_zip(download_zip,  to_dir)
    os.remove(download_zip)




#初始化一个浏览器（使用Chrome需安装chromedriver）
try:
    driver = webdriver.Chrome()
except Exception as e:
    download_driver(to_dir='./', version='76')
    driver = webdriver.Chrome()

time.sleep(1)

In [3]:
driver.get("https://kns.cnki.net/kns8/AdvSearch")
time.sleep(1)

search_words = '地理探测器'

search_middle = driver.find_element_by_class_name('search-middle')

dds = search_middle.find_elements_by_tag_name("dd")
dds[0].find_element_by_xpath('.//div[@class="sort reopt"]/div').click()
time.sleep(0.1)
dds[0].find_element_by_xpath('.//a[@value="AB"]').click()
time.sleep(0.1)
dds[0].find_element_by_tag_name('input').clear()
dds[0].find_element_by_tag_name('input').send_keys(search_words)
dds[0].find_element_by_xpath('.//div[@class="sort special"]').find_element_by_class_name('sort-default').click()
time.sleep(0.1)
dds[0].find_element_by_xpath('.//a[@value="="]').click()
time.sleep(0.1)
driver.find_element_by_class_name('search-buttons').click()

time.sleep(5)

In [10]:
html = driver.page_source
html = BeautifulSoup(html, 'lxml')
num_papers = int(html.find('span', {'class':'pagerTitleCell'}).em.text)
num_pages = int(html.find('span', {'class':'countPageMark'}).text.split('/')[1])
print('总共检索到 {} 篇文章, 共 {} 个页面'.format(num_papers, num_pages))

failed_list = []

总共检索到 921 篇文章, 共 47 个页面


In [12]:
end = False
while(not end):
    result_file = '搜索结果.csv'
    if os.path.exists(result_file):
        result = pd.read_csv(result_file, encoding='gbk')
    else:
        result = pd.DataFrame(columns=['论文ID','题名',  '作者', '来源', '发表时间', '数据库','被引', '专辑', '专题','分类号', '摘要', '关键词'])

    time.sleep(1)
    html = driver.page_source
    html = BeautifulSoup(html, 'lxml')
    
    try:
        current_page = html.find('span', {'class':'countPageMark'}).text
        print('正在爬取结果页面：{}\n'.format(current_page))
        table = html.find('table', {'class': 'result-table-list'}).findAll('tr')
    except:
        s = input('请在网页中输入验证码，或检查其他错误，待页面加载出来后，在此输入任意字符：')
        current_page = html.find('span', {'class':'countPageMark'}).text
        print('正在爬取结果页面：{}\n'.format(current_page))
        table = html.find('table', {'class': 'result-table-list'}).findAll('tr')

    for tr in table[1:]:
        try:
            time.sleep(0.5)
            td_name = tr.find('td',{'class':'name'})
            title = td_name.a.get_text().strip()
            dbcode = re.findall('DbCode=(.*?)[&|"]',str(td_name))[0]
            dbname = re.findall('DbName=(.*?)[&|"]',str(td_name))[0]
            filename = re.findall('FileName=(.*?)[&|"]',str(td_name))[0]
            url = 'https://kns.cnki.net/kcms/detail/detail.aspx?dbcode={}&dbname={}&filename={}'.format(dbcode,dbname,filename)
            authors = tr.find('td',{'class':'author'}).text
            source = tr.find('td',{'class':'source'}).text.strip()
            date = tr.find('td',{'class':'date'}).text.strip()
            data = tr.find('td',{'class':'data'}).text.strip()
            quote = tr.find('td',{'class':'quote'}).text.strip()
            detail_page = requests.get(url).text
            detail_page = BeautifulSoup(detail_page,'lxml')
            abstract = detail_page.find('span',{'class':'abstract-text'}).text
            keywords = ' '.join([i.strip() for i in detail_page.find('p',{'class':'keywords'}).text.split('\r\n')])
            div_doc = detail_page.find('div',{'class':'doc'})
            collection = div_doc.findAll(lambda tag: tag.get('class') is not None and 'top-space' in tag.get('class') and '专辑' in tag.text)[0].p.text
            topic = div_doc.findAll(lambda tag: tag.get('class') is not None and 'top-space' in tag.get('class') and '专题' in tag.text)[0].p.text
            class_id = div_doc.findAll(lambda tag: tag.get('class') is not None and 'top-space' in tag.get('class') and '分类号' in tag.text)[0].p.text
            result.loc[result.shape[0]] = {
                '论文ID'    : url,
                '题名'      : title,
                '作者'      : authors,
                '来源'      : source,
                '发表时间'  : date,
                '数据库'    : data,
                '被引'      : quote,
                '专辑'      : collection,
                '专题'      : topic,
                '分类号'    : class_id, 
                '摘要'      : abstract, 
                '关键词'    : keywords
            }
            print(result.iloc[-1],'\n')
        except Exception as e:
            print('获取信息失败：'+title)
            print('错误信息    ：',e)
            print()
            failed_list.append(title)
    
    result.to_csv(result_file, index=False, encoding='gbk')
    
    end = True
    page_links = driver.find_elements_by_xpath('//div[@class="pages"]/a')
    for a in page_links:
        if '下一页' in a.text:
            end=False
            a.click()


正在爬取结果页面：32/47

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                           基于地理探测器的天山北坡旅游空间分异特征及其影响因素研究
作者                                   梁巧霞; 黄杰; 谢霞; 李晓东; 江瞳
来源                                        西北师范大学学报(自然科学版)
发表时间                                           2018-11-15
数据库                                                    期刊
被引                                                      5
专辑                                          基础科学; 经济与管理科学
专题                                                     旅游
分类号                                                F592.7
摘要      以天山北坡为研究区,选取2000年、2005年、2010年、2015年新疆旅游数据,综合运用...
关键词                               天山北坡; 旅游; 空间分异; 地理探测器; 
Name: 531, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                 基于地理探测器和TOPSIS的县域农村空间贫困特征分析——以安徽省舒城县为例
作者                                      邓凯; 朱世泉; 黄长江; 袁慧慧
来源                                                 宿州学院学报
发表时间                         

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                  四川省山洪灾害时空分布规律及其影响因素研究
作者                                   熊俊楠;赵云亮;程维明;郭良;王楠;李伟
来源                                               地球信息科学学报
发表时间                                     2018-10-17 13:27
数据库                                                    期刊
被引                                                     12
专辑                                   基础科学; 工程科技Ⅰ辑; 工程科技Ⅱ辑
专题                                      安全科学与灾害防治; 水利水电工程
分类号                                              TV87;X43
摘要      山洪灾害时空分布规律及其影响因素,是灾害时空数据挖掘领域所关注的重点问题。本研究采用1950...
关键词                   山洪灾害; 时空分布规律; 标准差椭圆; 空间自相关; 地理探测器; 
Name: 542, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                    中国城市O3浓度时空变化特征及驱动因素
作者                                     黄小刚; 赵景波; 曹军骥; 宋永永
来源                                                   环境科学
发表时间                                     2018

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                       珠三角核心区农田耕层土壤有机碳库储量时空变化特征及其影响因素识别
作者                                          任向宁; 董玉祥; 王秋香
来源                                                   热带地理
发表时间                                     2018-09-06 17:02
数据库                                                    期刊
被引                                                       
专辑                                             基础科学; 农业科技
专题                                            农业基础科学; 农艺学
分类号                                                  S153
摘要      土壤有机碳库动态变化对区域乃至全球碳平衡具有显著影响,快速城市化地区社会经济迅速发展中土地利...
关键词            土壤有机碳; 农田耕层; 碳库储量; 地理探测器; 主导影响因素; 珠三角核心区; 
Name: 554, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                             大都市区农村土地非农化空间特征及机理——以广州市为例
作者                                      刘樱; 周春山; 黄婉玲; 朱倩琼
来源                                                 地理科学进展
发表时间                                     2018

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                  江苏省土地城镇化的空间分异及其主导因素探测
作者                                  吕立刚;隋雪艳;汪翔;周兵兵;孙燕;李永乐
来源                                                   人文地理
发表时间                                           2018-08-15
数据库                                                    期刊
被引                                                     11
专辑                                             文史哲; 经济与管理
专题                                           宏观经济管理与可持续发展
分类号                                               F299.23
摘要      实现新型城镇化战略的重要前提是科学认识土地城镇化的特征及机制,其难点在于识别影响土地城镇化的...
关键词                     土地城市化; 主导因素; 优化热点分析; 地理探测器; 江苏省; 
Name: 566, dtype: object 

正在爬取结果页面：34/47

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                  中国游客赴泰旅游流网络结构及其形成机理研究
作者                                          阮文奇; 张舒宁; 郑向敏
来源                                                 世界地理研究
发表时间                         

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                  贵州省地表温度的遥感反演评价及时空分异规律
作者                       胡泽银;王世杰;白晓永;李琴;吴路华;钱庆欢;肖建勇;陈飞;曾成
来源                                                  生态学杂志
发表时间                                     2018-07-04 18:05
数据库                                                    期刊
被引                                                      2
专辑                               基础科学; 农业科技; 工程科技Ⅱ辑; 信息科技
专题                                       工业通用技术及设备; 自动化技术
分类号                                                  TP79
摘要      地表温度（LST）是全球变化研究中的关键参数,对生态系统和生物地球化学等具有重要研究意义。然...
关键词                              地表温度; 遥感反演; 时空分异; 单峰分布; 
Name: 577, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                 岷江上游藏羌聚落生境分异与气候变化的关联机制
作者                                                    郭亚琳
来源                                                 西南科技大学
发表时间                                         

...

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                            基于信息图谱的吉林省西部湿地空间格局演变及驱动机制研究
作者                                                    赵丹丹
来源                                                 吉林师范大学
发表时间                                           2016-06-01
数据库                                                    硕士
被引                                                       
专辑                                         理工A(数学物理力学天地生)
专题                                   自然地理学和测绘学; 自然地理学和测绘学
分类号                                             P237;P901
摘要      吉林省西部地处我国北方农牧交错带的东段,位于中国湿润的东部季风和内陆干旱、半干旱的过渡带,属...
关键词            地学信息图谱; 地理探测器; CA-Markov模型; 空间自相关; 吉林省西部; 
Name: 736, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                              基于地理探测器模型的自然和人为因素对植被变化的影响
作者                                                     庞静
来源                                                   山西大学
发表时间                                         

获取信息失败：Spatial association between dissection density and environmental factors over the entire conterminous United States
错误信息    ： list index out of range

获取信息失败：Spatial association between dissection density and environmental factors over the entire conterminous United States
错误信息    ： list index out of range

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                  一种基于地理探测器的城镇扩展影响机理分析法
作者                                               徐秋蓉; 郑新奇
来源                                                   测绘学报
发表时间                                           2015-12-15
数据库                                                    期刊
被引                                                     36
专辑                                         理工A(数学物理力学天地生)
专题                                              自然地理学和测绘学
分类号                                                  P204
摘要      城市化是全球范围内普遍存在的重要现象,尤其发展中国家和地区正经历着快速城市化的过程。快速城市...
关键词                             城镇扩展; 地理探测器; 移

获取信息失败：Economic Growth Factor Detection of National-Level Economic and Technological Development Zones in China
错误信息    ： list index out of range

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                地理探测器在史前聚落人地关系研究中的应用与分析
作者                                  毕硕本; 计晗; 陈昌春; 杨鸿儒; 沈香
来源                                                 地理科学进展
发表时间                                     2015-01-26 11:49
数据库                                                    期刊
被引                                                     44
专辑                                    理工A(数学物理力学天地生); 文史哲
专题                                          自然地理学和测绘学; 考古
分类号                                             K878;P208
摘要      遗址—河流距离是史前聚落遗址人地关系研究的重要内容。本文以河南省卢氏县为例,引入地理探测器模...
关键词                          史前聚落; 遗址—河流距离; 人地关系; 地理探测器; 
Name: 758, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                  浙江省区域创新平台空间分布特征及其影响因素
作者            

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                   东北地区资源型城市转型绩效及影响因素研究
作者                                      谭俊涛; 张平宇; 李静; 刘世薇
来源                         2015年中国地理学会经济地理专业委员会学术研讨会论文摘要集
发表时间                                           2015-06-26
数据库                                                  中国会议
被引                                                      4
专辑                                                  经济与管理
专题                                           宏观经济管理与可持续发展
分类号                                               F299.27
摘要      从经济、社会和生态环境三个方面,构建资源型城市可持续发展能力评价指标体系,运用可持续发展能力...
关键词                          资源型城市; 经济转型; 绩效评估; 东北老工业基地; 
Name: 769, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                基于Whittaker滤波的陕西省植被物候特征
作者                                                张晗; 任志远
来源                                                   中国沙漠
发表时间                                     2015

获取信息失败：Spatial-temporal characteristics and impact factors of newly increased farmland by land consolidation in Hubei province at county level
错误信息    ： list index out of range

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                             我国流行性乙型脑炎时空分布特征及相关地理气候因素研究
作者                                                    李晓龙
来源                                             中国疾病预防控制中心
发表时间                                           2014-06-30
数据库                                                    硕士
被引                                                      5
专辑                                                 医药卫生科技
专题                                        感染性疾病及传染病; 神经病学
分类号                                               R512.32
摘要      流行性乙型脑炎(Japanese encephalitis,以下简称乙脑)是感染乙脑病毒(J...
关键词                   流行性乙型脑炎; 时空分布特征; 地貌单元; 气候因素; 相关分析; 
Name: 780, dtype: object 

论文ID    https://kns.cnki.net/kcms/detail/detail.aspx?d...
题名                                   近60年

In [15]:
result = result.drop_duplicates(keep='first')  # 去重

In [18]:
result.to_csv(result_file, index=False, encoding='gbk')